In [ ]:
from scipy.fft import fft, fftfreq
import sounddevice
from scipy.io.wavfile import write, read
num_secs = 4
wav = read("sounds/gc.wav")
sample_rate = wav[0]
wav = wav[1][:num_secs* sample_rate]

low_pass = 300
high_pass = 2000

In [54]:
#filter try
import scipy.signal 
def lowpass(data: np.ndarray, cutoff: float, sample_rate: float, poles: int = 5):
    sos = scipy.signal.butter(poles, cutoff, 'lowpass', fs=sample_rate, output='sos')
    filtered_data = scipy.signal.sosfiltfilt(sos, data)
    return filtered_data
def highpass(data: np.ndarray, cutoff: float, sample_rate: float, poles: int = 5):
    sos = scipy.signal.butter(poles, cutoff, 'highpass', fs=sample_rate, output='sos')
    filtered_data = scipy.signal.sosfiltfilt(sos, data)
    return filtered_data
def bandpass(data: np.ndarray, cutoff, sample_rate: float, poles: int = 5):
    sos = scipy.signal.butter(poles, cutoff, 'bandpass', fs=sample_rate, output='sos')
    filtered_data = scipy.signal.sosfiltfilt(sos, data)
    return filtered_data
sounddevice.play(wav, samplerate=sample_rate, blocking= True)
new_wav = highpass(wav, low_pass,sample_rate=sample_rate)
oth_new_wav = lowpass(wav, high_pass,sample_rate=sample_rate)
final_new_wav = bandpass(wav, [low_pass, high_pass], sample_rate=sample_rate)
sounddevice.play(new_wav, samplerate=sample_rate)
#new_wav = np.add(new_wav, oth_new_wav)


In [45]:
import matplotlib.pyplot as plt
import numpy as np
# need to window and have windowing function
fft_wav = wav
final_wav = np.array([])
for block in range(0,len(fft_wav),1024):
    #block 
    tmp_wav = fft_wav[block:block + 1024]
    fft_values =  np.absolute(fft(tmp_wav))

    freq = [i for i, f in enumerate(fftfreq(len(tmp_wav), 1/sample_rate)) if f > 0]
    fft_values = [fft_values[f]  for i,f in enumerate(freq) ]

    low_bins = [i for i,f in enumerate(freq) if f < low_pass and f > 0]
    low_power = np.average([fft_values[i] for i in low_bins])
    mid_bins = [i for i,f in enumerate(freq) if f > low_pass and f < high_pass]
    mid_power = np.average([fft_values[i] for i in mid_bins])

    high_bins = [i for i,f in enumerate(freq) if f > high_pass]
    high_power = np.average([fft_values[i] for i in high_bins])
    min_power = min(low_power, mid_power, high_power)
    low_ratio = min_power / low_power  
    mid_ratio = min_power / mid_power  
    high_ratio = min_power / high_power  
    #print(f"low: {low_ratio}")
    #print(f"mid: {mid_ratio}")
    #print(f"high: {high_ratio}")
    low_wav = highpass(wav, low_pass,sample_rate=sample_rate)
    high_wav = lowpass(wav, high_pass,sample_rate=sample_rate)
    mid_wav = bandpass(wav, [low_pass, high_pass], sample_rate=sample_rate)
    low_wav *= low_ratio
    mid_wav *= mid_ratio
    high_wav *= high_ratio
    out_wav = np.add(high_wav, np.add(low_wav, mid_wav) )
    np.append(final_wav, out_wav)

print(len(final_wav), len(wav))
sounddevice.play(final_wav, samplerate=sample_rate)
    #plt.plot(freq,fft_values)
    #plt.show()
    #magnitude = np.absolute(fft_values)
    #print(magnitude)
#phase = np.angle(fft_values[index[0]])
#print("Magnitude:", magnitude, ", phase:", phase)

## Plot a spectrum 
#plt.plot(freq[0:sample_rate//2], 2/sample_rate*np.abs(fft_values[0:sample_rate//2]), label='amplitude spectrum')   # in a conventional form
#plt.plot(freq[0:sample_rate//2], np.angle(fft_values[0:sample_rate//2]), label='phase spectrum')
#plt.legend()
#plt.grid()
#plt.show()

KeyboardInterrupt: 